# Actividad 4: Selección del mejor modelo de ML

Durante la etapa de entrenamiento nos podemos encontrar con diferentes estrategias de cara a mejorar el desempeño del modelo. Una de estas estrategias es conocida como **Model Selection**, en la cual, probamos con diferentes algoritmos de Machine Learning y al final, escogemos el mejor modelo con base al que obtenga las mejores métricas. Para esta actividad, seguiremos con el mismo caso de uso descrito en la Actividad 3, que es la clasificación de cancer de mama entre los tipos **Maligno y Benigno**.

En la pasada actividad usamos SVM como clasificador, en esta Activdad 3, evaluaremos el desempeño del modelo al usar 3 clasificadores diferentes, que son: **LightGBM, MLPClassifier (perceptrón multicapa) y Logistic Regression**. Al final, tomaremos el mejor modelo con base en el mejor **Accuracy**

# Imports 

Librerias que se necesitan para el desarrollo de la actividad

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lightgbm as ltb
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Configurando pandas para visualización de 100 columnas

In [2]:
pd.options.display.max_columns = 100

# Lectura del dataset

In [3]:
path = 'https://raw.githubusercontent.com/jjguajo/activities_ai/bf35e2d054c00d10367e4741bbec2f45251084ff/data_activity_3_ml.csv'
data = pd.read_csv(path, on_bad_lines='skip')

# Entrenamiento del modelo

Lo descrito en esta sección es reutilizado de la Actividad anterior como base para la evaluación de diferentes clasificadores.

In [4]:
features_finales = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean", 
    "fractal_dimension_mean", "diagnosis"
]

In [5]:
# Seleccionamos solo las features de interés del dataset completo
dataset_final = data[features_finales]

In [6]:
# Variable categórica creada a partir de la feature fractal_dimension_mean. 
dataset_final["calc_fractal_dimension_mean"] = np.where(
    1, dataset_final["fractal_dimension_mean"] < 0.055, 
    0
)

<ipython-input-6-2ddb9cafa4af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_final["calc_fractal_dimension_mean"] = np.where(


In [7]:
# Eliminamos la variable númerica que usamos para crear la variable categórica
dataset_final.drop(['fractal_dimension_mean'], axis=1, inplace=True)

<ipython-input-7-b89a3607ea7f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_final.drop(['fractal_dimension_mean'], axis=1, inplace=True)


In [8]:
# Clase que realiza el fit y transform a una variable
class ColumnSelector(BaseEstimator, TransformerMixin):
    """Select only specified columns."""
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]

In [9]:
# Features numéricas
numerical = [
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave points_mean", "symmetry_mean"
]

In [10]:
# ColumnSelector: selecciona las columnas númericas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con la mediana de todos los valores de la feature
num_pipe = Pipeline([
    ('selector', ColumnSelector(numerical)),
    ('imputer', SimpleImputer(strategy='median'))
])

In [11]:
# Features categóricas
categorical = ["calc_fractal_dimension_mean"]

In [12]:
# ColumnSelector: selecciona las columnas categóricas del dataset de entrenamiento
# SimpleImputer: los valores faltantes los completa con un valor fijo
# OneHotEncoder: codificación de variables categóricas
cat_pipe = Pipeline([
    ('selector', ColumnSelector(categorical)),
    ('imputer', SimpleImputer(strategy='constant')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])

In [13]:
# Feature union de todas las features categóricas y numéricas
preprocessor = FeatureUnion([
    ('cat', cat_pipe),
    ('num', num_pipe)
])

In [14]:
# Se elimina la variable target de los datos que contienen las features 
X = dataset_final.drop(['diagnosis'], axis = 1)

In [15]:
# Se genera en una variable separada el target del problema, en este caso "Diagnosis"
y = dataset_final['diagnosis']

In [16]:
# De todo el conjunto de datos se toman crean datos de train y test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)
print("Size of training set:", X_train.shape)
print("Size of test set:", X_test.shape)

Size of training set: (398, 10)
Size of test set: (171, 10)


# Evaluación de LightGBM

In [17]:
# Instalación LightGBM
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
# Pipeline 
pipe_lightgbm = Pipeline([
    ('preprocessor', preprocessor),
    ('model', ltb.LGBMClassifier())
])

In [19]:
# Entrenamiento
model_lightgbm = pipe_lightgbm.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [20]:
# Predicción sobre los datos de test
y_pred = model_lightgbm.predict(X_test)

In [21]:
# Classification report de resultados al usar LightGBM
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.95      0.98      0.97       108
           M       0.97      0.92      0.94        63

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171



*Se puede observar que en un primera iteración, al usar LightGBM se obtiene un mejor accuracy respecto a lo obtenido en la actividad anterior usando SVM. A partir de esto, ya queda a criterio del científico de datos hacer uso de búsqueda de hiperaparámetros. Este paso tambien depende si la métrica obtenida es igual o mayor a la superior. Para esta actividad, tomaremos el accuracy mas alto*

# Evaluación de MLPClassifier

In [22]:
# Pipeline
pipe_mlp = Pipeline([
    ('preprocessor', preprocessor),
    ('model', MLPClassifier())
])

In [23]:
# Entrenamiento
model_mlp = pipe_mlp.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [24]:
# Predicción sobre los datos de test
y_pred = model_mlp.predict(X_test)

In [25]:
# Classification report de resultados al usar MLPClassifier
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.90      0.95      0.93       108
           M       0.91      0.83      0.87        63

    accuracy                           0.91       171
   macro avg       0.91      0.89      0.90       171
weighted avg       0.91      0.91      0.91       171



Se puede observar que se obtiene un accuracy del 0.91, la cual es menor que la obtenida por LightGBM y igual a la obtenida en la actividad anterior. Haremos una optimización de hiperaparámetros para intentar aumentar esta métrica

In [26]:
# Rango de hiperparámetros
mlp_gs = MLPClassifier(max_iter=100)
parameter_space = {
    'hidden_layer_sizes': [(10,30,10),(20,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive']
}

In [27]:
# Búsqueda de los mejores hiperparámetros
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(


GridSearchCV(cv=5, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(10, 30, 10), (20,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [28]:
# Hipaparámetros encontrados por GirdSearch para MLPClassifier
clf.best_params_

{'activation': 'relu',
 'alpha': 0.0001,
 'hidden_layer_sizes': (10, 30, 10),
 'learning_rate': 'constant',
 'solver': 'adam'}

In [29]:
# Instanciamento del modelo con los hiperparámetros encontrados
clf = MLPClassifier(**clf.best_params_)

In [30]:
# Pipeline con nuevos hiperparámetros
pipe_mlp = Pipeline([
    ('preprocessor', preprocessor),
    ('model', clf)
])

In [31]:
# Entrenamiento
model_mlp = pipe_mlp.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [32]:
# Predicción sobre los datos de test
y_pred = model_mlp.predict(X_test)

In [ ]:
# Métricas con hiperparámetros encontrados
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.88      0.98      0.93       108
           M       0.96      0.76      0.85        63

    accuracy                           0.90       171
   macro avg       0.92      0.87      0.89       171
weighted avg       0.91      0.90      0.90       171



Podemos observar que en términos de accuracy, se obtiene un valor menor a los obtenidos en iteraciones anteriores.

# Evaluación de Logistic Regression

In [33]:
# Pipeline final del proceso de entrenamiento
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])

In [34]:
# Entrenamiento
model_lr = pipe_lr.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [35]:
# Predicción sobre los datos de test
y_pred = model_lr.predict(X_test)

In [36]:
# Classification report de resultados al usar LogisticRegression
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.93      0.94      0.93       108
           M       0.89      0.87      0.88        63

    accuracy                           0.91       171
   macro avg       0.91      0.90      0.91       171
weighted avg       0.91      0.91      0.91       171



También, se puede observar que se obtiene un accuracy del 0.91, la cual es menor que la obtenida por LightGBM y igual a la obtenida en la actividad anterior. Nuevamente, haremos una optimización de hiperaparámetros para intentar aumentar esta métrica.

In [37]:
# Rango de hiperparámetros
parameters = {
    "C":np.logspace(-3,3,7), 
    "penalty":["l2"], 
    "solver":['liblinear','newton-cg']
}

In [38]:
# Búsqueda de los mejores hiperparámetros
clf = GridSearchCV(LogisticRegression(), param_grid = parameters, cv = 5, verbose=True, n_jobs=-1)
clf.fit(X_train, y_train)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l2'],
                         'solver': ['liblinear', 'newton-cg']},
             verbose=True)

In [39]:
# Hiperparámetros encontrados
clf.best_params_

{'C': 1000.0, 'penalty': 'l2', 'solver': 'newton-cg'}

In [40]:
# Pipeline con nuevos hiperparametros
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(**clf.best_params_))
])

In [41]:
# Entrenamiento
model_lr = pipe_lr.fit(X_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [42]:
# Predicción sobre los datos de test
y_pred = model_lr.predict(X_test)

In [43]:
# Métricas con nuevos hiperparametros
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           B       0.94      0.96      0.95       108
           M       0.93      0.89      0.91        63

    accuracy                           0.94       171
   macro avg       0.94      0.93      0.93       171
weighted avg       0.94      0.94      0.94       171



Se puede observar, que para LogisticRegression se obtiene un mayor accuracy, 0.94. Es un accuracy alto, sin embargo, es menor al obtenido por LightGBM. 